In [ ]:
import os
for root, dirs, files in os.walk("c:\\users\\user\\documents\\cta_seg"):
    for file in files:
        with open(os.path.join(root, file), "r", encoding="cp1251") as f:
            text = f.read()
        with open(os.path.join("c:\\users\\user\\documents\\cta_seg_utf8", file), "w", encoding="utf-8") as f:
            f.write(text)

In [ ]:
import glob
for i in glob.glob("**/*.txt", recursive=True):
    print(i)

KeyboardInterrupt: ignored

In [ ]:
a = "a,b,c".split(",")
a[0], a[2]

('a', 'c')

Практическое задание:
1. Обработать все файлы .seg_* в директории
2. Извлечь информацию о звуках и словах
3. Сопоставить каждому слову транскрипцию
4. Занести информацию в таблицу:

Слово | Сколько раз встретилось | Транскрипция 1 | Сколько раз встретилась | Транскрипция 2 | Сколько раз встретилась | ... 

5. Сортировать по колонке 2 по убыванию

#Составление словаря

Получить для каждого слова все варианты его транскрипции. Из тех, которые оканчиваются на звонкие (если нет варианта, заканчивающегося на глухой, то создать такой вариант, иначе - убрать).
Большие буквы - звонкие аллофоны Ц, Ч. Апостроф - мягкость, подчеркивание - твердость. Если слово заканчивается на более, чем один согласный, то они все озвончаются.

In [117]:
from glob import glob
import pickle
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def detect_encoding(file_path):
    encoding = "utf-8"
    try:
        l = open(file_path, 'r', encoding="utf-8").read()
        if l.startswith("\ufeff"):
            encoding = "utf-8-sig"
    except UnicodeDecodeError:
        try:
            open(file_path, 'r', encoding="utf-16").read()
            encoding = "utf-16"
        except UnicodeError:
            encoding = "cp1251"
    return encoding

In [5]:
def read_seg(path):
  with open(path, "r", encoding=detect_encoding(path)) as f:
    lines = [i.strip() for i in f.readlines()]
  return lines

In [79]:
def read_txt(path):
  with open(path, "r", encoding=detect_encoding(path)) as f:
    lines = [i.split() for i in f.readlines()]
  return lines

In [6]:
path = "/content/drive/MyDrive/STUDY_MY/Programming/cta_seg"

In [7]:
seg_files_words = sorted(glob(path+'/*.seg_Y1')) # читаем сеги со словами

In [8]:
read_seg(seg_files_words[0])

['[PARAMETERS]',
 'SAMPLING_FREQ=22050',
 'BYTE_PER_SAMPLE=2',
 'CODE=0',
 'N_CHANNEL=1',
 'N_LABEL=3',
 '[LABELS]',
 '0,8,юрий',
 '18180,8,трифонов',
 '54662,8,']

In [9]:
seg_files_sounds = sorted(glob(path+'/*.seg_B1')) # читаем сеги со звуками

In [10]:
read_seg(seg_files_sounds[0])

['[PARAMETERS]',
 'SAMPLING_FREQ=22050',
 'BYTE_PER_SAMPLE=2',
 'CODE=0',
 'N_CHANNEL=1',
 'N_LABEL=13',
 '[LABELS]',
 '0,2,j',
 '8492,2,u0',
 "12708,2,r'",
 '13708,2,i4',
 '18180,2,t',
 "24904,2,r'",
 '25940,2,i0',
 '31544,2,f',
 '36806,2,a4',
 '38604,2,n',
 '41618,2,a4',
 '44508,2,f',
 '54662,2,']

In [11]:
seg_tuples = [] # собираем сеги в тюплы, чтобы они относилилсь оба к одному и тому же звуковому файлу
for word in seg_files_words:
  name = word.split('.')[0]
  if seg_files_sounds.count(f'{name}.seg_B1'):
    seg_tuples.append((word, f'{name}.seg_B1'))
    print(seg_tuples[-1])

('/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0001.seg_Y1', '/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0001.seg_B1')
('/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0002.seg_Y1', '/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0002.seg_B1')
('/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0003.seg_Y1', '/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0003.seg_B1')
('/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0004.seg_Y1', '/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0004.seg_B1')
('/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0005.seg_Y1', '/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0005.seg_B1')
('/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0006.seg_Y1', '/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0006.seg_B1')
('/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0007.seg_Y1', '/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/cta0007.seg_B1')
('/content/dr

In [26]:
def get_keys_n_sounds(words_path, sounds_path, channels=2): # функция соотносящая словам в файле - ключам массивы значений значков транскрипции подходящие по времени
  words = [i for i in read_seg(words_path)[7:]]
  for i, v in enumerate(words):
    words[i] = words[i].replace("[", '')
    words[i] = words[i].replace("]", '')
    words[i] = words[i].replace("-", '')
    words[i] = words[i].replace("+", '')
  sounds = [i for i in read_seg(sounds_path)[7:]]
  keys = [i.split(',')[2] for i in words]
  value_times = [i.split(',')[0] for i in sounds]
  dictionary_words_to_sounds = dict.fromkeys(keys, [])
  for k in range(len(dictionary_words_to_sounds.keys()) - 1):
    word_time = (words[k].split(',')[0], words[k + 1].split(',')[0])
    start_ind = -1
    end_ind = -1
    for s in range(1, len(sounds)):
      if start_ind == -1 and int(word_time[0]) >= int(value_times[s - 1]) and int(word_time[0]) < int(value_times[s]):
        start_ind = s - 1
      if start_ind == -1 and int(word_time[0]) == int(value_times[s]):
        start_ind = s
      if end_ind == -1 and int(word_time[1]) > int(value_times[s - 1]) and int(word_time[1]) <= int(value_times[s]):
        end_ind = s
      if end_ind == -1 and int(word_time[1]) == int(value_times[s - 1]):
        end_ind = s - 1
    symbols = [i.split(',')[2] for i in sounds[start_ind:end_ind]]
    dictionary_words_to_sounds[keys[k]] = dictionary_words_to_sounds.get(keys[k], []) + [symbols]
  return dictionary_words_to_sounds

In [27]:
def get_keys_n_sounds_from_all(seg_tuples): # функция, прогоняющая по всему тюплу путей файлов функцию сверху и объединяющая значения словарей
  first_dict = get_keys_n_sounds(*seg_tuples[0])
  merged_dict = {**first_dict}
  for i in range(1, len(seg_tuples)):
    print(i)
    merged_dict = merged_dict | get_keys_n_sounds(*seg_tuples[i])
  return merged_dict

In [28]:
get_keys_n_sounds(*seg_tuples[0]) # пример

{'юрий': [['j', 'u0', "r'", 'i4']],
 'трифонов': [['t', "r'", 'i0', 'f', 'a4', 'n', 'a4', 'f']],
 '': []}

In [29]:
res_dict = get_keys_n_sounds_from_all(seg_tuples)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [30]:
res_dict # по каким-то причинам некоторые слова вообще не нашлись...

{'юрий': [['j', 'u0', "r'", 'i4']],
 'трифонов': [['t', "r'", 'i0', 'f', 'a4', 'n', 'a4', 'f']],
 '': [],
 'обмен': [['a1', 'b', "m'", 'e0', 'n']],
 'в': [['v']],
 'июле': [['y1', 'j', 'u0', "l'", 'e4']],
 'мать': [['m', 'a0', "t'"]],
 'дмитриева': [['d', "m'", 'i0', 't', "r'", 'i4', 'i4', 'v', 'a4']],
 'ксения': [['k', "s'", 'e0', "n'", 'i4', 'i4']],
 'федоровна': [],
 'тяжело': [["t'", 'i1', 'zh', 'y1', 'l', 'o0']],
 'заболела': [['z', 'a2', 'b', 'a1', "l'", 'e0', 'l', 'a4']],
 'и': [['i1']],
 'ее': [['j', 'i1', 'j', 'o0']],
 'отвезли': [['a1', 't', "v'", 'i1', 'z', "l'", 'i0']],
 'боткинскую': [['b', 'o0', 't', "k'", 'i4', 'n', 's', 'k', 'u4', 'j', 'u4']],
 'где': [],
 'она': [['a1', 'n', 'a0']],
 'пролежала': [['p', 'r', 'a2', "l'", 'i1', 'zh', 'a0', 'l', 'a4']],
 'двенадцать': [['d', "v'", 'i1', 'n', 'a0', 'c', 'y4', "d'"]],
 'дней': [['d', "n'", 'e0', 'i4']],
 'с': [],
 'подозрением': [],
 'на': [['n', 'a1']],
 'самое': [['s', 'a0', 'm', 'a4', 'e4']],
 'худшее': [['h', 'u0', 't',

In [35]:
len(res_dict)

5168

In [32]:
cleaned_res_dict = dict(res_dict) # убрал варианты без транскрипции
for k, v in res_dict.items():
  if v == []:
    cleaned_res_dict.pop(k)

In [34]:
len(cleaned_res_dict)

4167

In [36]:
with open(path+"/"+'our_dict.pickle', 'wb') as handle: # сохранил словарь в пикл, чтобы заново не прогонять
    pickle.dump(cleaned_res_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(path+"/"+'our_dict.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(cleaned_res_dict == b)

True


In [37]:
with open(path+"/"+'our_dict.pickle', 'rb') as handle: # достал словарь из пикла
    res_dict = pickle.load(handle)

In [38]:
res_dict

{'юрий': [['j', 'u0', "r'", 'i4']],
 'трифонов': [['t', "r'", 'i0', 'f', 'a4', 'n', 'a4', 'f']],
 'обмен': [['a1', 'b', "m'", 'e0', 'n']],
 'в': [['v']],
 'июле': [['y1', 'j', 'u0', "l'", 'e4']],
 'мать': [['m', 'a0', "t'"]],
 'дмитриева': [['d', "m'", 'i0', 't', "r'", 'i4', 'i4', 'v', 'a4']],
 'ксения': [['k', "s'", 'e0', "n'", 'i4', 'i4']],
 'тяжело': [["t'", 'i1', 'zh', 'y1', 'l', 'o0']],
 'заболела': [['z', 'a2', 'b', 'a1', "l'", 'e0', 'l', 'a4']],
 'и': [['i1']],
 'ее': [['j', 'i1', 'j', 'o0']],
 'отвезли': [['a1', 't', "v'", 'i1', 'z', "l'", 'i0']],
 'боткинскую': [['b', 'o0', 't', "k'", 'i4', 'n', 's', 'k', 'u4', 'j', 'u4']],
 'она': [['a1', 'n', 'a0']],
 'пролежала': [['p', 'r', 'a2', "l'", 'i1', 'zh', 'a0', 'l', 'a4']],
 'двенадцать': [['d', "v'", 'i1', 'n', 'a0', 'c', 'y4', "d'"]],
 'дней': [['d', "n'", 'e0', 'i4']],
 'на': [['n', 'a1']],
 'самое': [['s', 'a0', 'm', 'a4', 'e4']],
 'худшее': [['h', 'u0', 't', 'sh', 'e4']],
 'сентябре': [["s'", 'i1', 'n', "t'", 'i1', 'b', "r'",

In [39]:
VOIC_UNVOIC_CONSONANTS = [
    ("b", "p"),
    ("b\'", "p\'"),
    ("v", "f"),
    ("v\'", "f\'"),
    ("g", "k"),
    ("g\'", "k\'"),
    ("d", "t"),
    ("d\'", "t\'"),
    ("z", "s"),
    ("z\'", "s\'"),
    ("zh", "sh"),
    ("Ts", "ts"),
    ("X", "x"),
    ("X\'", "x\'"),
    ("Ch", "ch"),
    ("Sh\'", "sh\'"),
] # массив пар звонких и глухих согласных

In [40]:
def change_last_sound_if_voiced_consonant(words_and_sounds_dict): # функция, проверяющая 1, 2 конечных согласных на то, что они звонкие или глухие, и меняющая их на глухие, если они оказались звонкими, либо убирающая некорректные варианты
  voic = [i[0] for i in VOIC_UNVOIC_CONSONANTS]
  unvoic = [i[1] for i in VOIC_UNVOIC_CONSONANTS]
  for k, v in words_and_sounds_dict.items():
    if type(k) == list:
      print(k)
      continue
    i = -1
    ends_with_consonant = False
    ends_with_consonants = False
    is_there_right_variant = False
    var_with_unvoiced_l_cons = []
    for variant in v:
      if variant[-1] in voic or variant[-1] in unvoic:
        ends_with_consonant = True
        if variant[-1] in unvoic:
           is_there_right_variant = True
           var_with_unvoiced_l_cons.append(variant)
        if len(variant) > 1:
          if variant[-2] in voic or variant[-2] in unvoic:
            ends_with_consonants = True
    if ends_with_consonant:
      if len(v) == 1:
        if is_there_right_variant:
          print('1', v)
          continue
        else:
          print('2', v)
          if not ends_with_consonants:
            words_and_sounds_dict[k][0][-1] = unvoic[voic.index(v[0][-1])] # заменяем звонкий на соответствующий ему глухой
          else:
            words_and_sounds_dict[k][0][-1] = unvoic[voic.index(v[0][-1])] # заменяем звонкий на соответствующий ему глухой
            words_and_sounds_dict[k][0][-2] = unvoic[voic.index(v[0][-2])] # заменяем звонкий на соответствующий ему глухой
      else:
        if is_there_right_variant:
          print('3', v)
          words_and_sounds_dict[k] = [var for var in v if var in var_with_unvoiced_l_cons][0] # удаляем варианты без конечной глухой согласной
        else:
          print('4', v)
          if not ends_with_consonants:
            words_and_sounds_dict[k] = [var[:len(var) - 2] + unvoic[voic.index(var[-1])] for var in v][0] # создаем варианты с конечной глухой согласной
          else:
            words_and_sounds_dict[k] = [var[:len(var) - 3] + unvoic[voic.index(var[-2])] + unvoic[voic.index(var[-1])] for var in v][0] # создаем варианты с конечной глухой согласной
  for k, v in words_and_sounds_dict.items():
    words_and_sounds_dict[k] = v[0]

  return words_and_sounds_dict

In [41]:
new_res_dict = change_last_sound_if_voiced_consonant(res_dict) # занятно, но судя по всему здесь нет варианта, когда надо заменить конечную согласную...

1 [['t', "r'", 'i0', 'f', 'a4', 'n', 'a4', 'f']]
2 [['v']]
1 [['m', 'a0', "t'"]]
2 [['d', "v'", 'i1', 'n', 'a0', 'c', 'y4', "d'"]]
1 [['p', 'y1', 't', "v'", 'i1', 'r', "d'", 'i0', 'l', 'y4', "s'"]]
1 [['h', 'a1', "d'", 'i0', "t'"]]
1 [['v', 'o0', 't']]
1 [["v'", 'e1', 'r', 'n', 'u0', 'l', 'e4', "s'"]]
1 [['d', "m'", 'i0', 't', "r'", 'i4', 'i4', 'f']]
1 [['r', 'a0', 's']]
1 [['a1', 'ch', 'i1', 'k', 'a0', "n'", 'i4', "l'", 'i4', "s'"]]
1 [["l'", 'e0', 't']]
1 [["l'", 'i1', 'sh']]
1 [['p', "r'", 'i1', 'v', 'r', 'a1', "t'", 'i0', 'l', 'y4', "s'"]]
1 [['h', 'a1', "t'", 'e0', 'l', 'y4', "s'"]]
1 [['u1', 'k', 'a1', 'l', 'o0', "t'"]]
1 [['a1', 'b', "v'", 'i1', "n'", 'i0', "t'"]]
1 [['h', 'o0', 'ch', 'i4', 'sh']]
1 [['zh', 'y0', "t'"]]
1 [['p', 'a1', 't', "r'", 'e0', 'b', 'n', 'y4', "s'"]]
1 [["s'", 'j', 'i1', 'z', "v'", 'i0', "t'"]]
1 [['n', 'a2', 'd', 'a1', "v'", 'i0', "t'"]]
1 [['k', 'a1', 'k', 'o0', 'v', 'y4', "n'", 'i1', 'b', 'u0', "t'"]]
1 [['s', 'v', 'a1', "d'", 'i0', "t'"]]
1 [["m'", 'i

In [42]:
new_res_dict

{'юрий': ['j', 'u0', "r'", 'i4'],
 'трифонов': ['t', "r'", 'i0', 'f', 'a4', 'n', 'a4', 'f'],
 'обмен': ['a1', 'b', "m'", 'e0', 'n'],
 'в': ['f'],
 'июле': ['y1', 'j', 'u0', "l'", 'e4'],
 'мать': ['m', 'a0', "t'"],
 'дмитриева': ['d', "m'", 'i0', 't', "r'", 'i4', 'i4', 'v', 'a4'],
 'ксения': ['k', "s'", 'e0', "n'", 'i4', 'i4'],
 'тяжело': ["t'", 'i1', 'zh', 'y1', 'l', 'o0'],
 'заболела': ['z', 'a2', 'b', 'a1', "l'", 'e0', 'l', 'a4'],
 'и': ['i1'],
 'ее': ['j', 'i1', 'j', 'o0'],
 'отвезли': ['a1', 't', "v'", 'i1', 'z', "l'", 'i0'],
 'боткинскую': ['b', 'o0', 't', "k'", 'i4', 'n', 's', 'k', 'u4', 'j', 'u4'],
 'она': ['a1', 'n', 'a0'],
 'пролежала': ['p', 'r', 'a2', "l'", 'i1', 'zh', 'a0', 'l', 'a4'],
 'двенадцать': ['d', "v'", 'i1', 'n', 'a0', 'c', 'y4', "t'"],
 'дней': ['d', "n'", 'e0', 'i4'],
 'на': ['n', 'a1'],
 'самое': ['s', 'a0', 'm', 'a4', 'e4'],
 'худшее': ['h', 'u0', 't', 'sh', 'e4'],
 'сентябре': ["s'", 'i1', 'n', "t'", 'i1', 'b', "r'", 'e0'],
 'сделали': ["z'", "d'", 'e0', 'l',

In [62]:
with open(path+'/lexicon.txt', 'w', encoding="utf-8") as fl:
    for k, v in new_res_dict.items():
      fl.write(k + ' ' + " ".join(v) +"\n") #[l if '1' in l and '4' in l == False else l[0] for l in v]) +"\n")

In [63]:
with open(path+'/lexicon.txt', "r", encoding=detect_encoding(path + 'lexicon.txt')) as f:
  lines = [i.strip() for i in f.readlines()]
  print(lines)

["юрий j u0 r' i4", "трифонов t r' i0 f a4 n a4 f", "обмен a1 b m' e0 n", 'в f', "июле y1 j u0 l' e4", "мать m a0 t'", "дмитриева d m' i0 t r' i4 i4 v a4", "ксения k s' e0 n' i4 i4", "тяжело t' i1 zh y1 l o0", "заболела z a2 b a1 l' e0 l a4", 'и i1', 'ее j i1 j o0', "отвезли a1 t v' i1 z l' i0", "боткинскую b o0 t k' i4 n s k u4 j u4", 'она a1 n a0', "пролежала p r a2 l' i1 zh a0 l a4", "двенадцать d v' i1 n a0 c y4 t'", "дней d n' e0 i4", 'на n a1', 'самое s a0 m a4 e4', 'худшее h u0 t sh e4', "сентябре s' i1 n t' i1 b r' e0", "сделали z' d' e0 l y4 l' e4", "операцию a1 p' e1 r a0 c y4 y4", "подтвердилось p y1 t v' i1 r d' i0 l y4 s'", 'но n o1', 'считавшая sc i1 t a0 f sh e4 e4', 'что sh t o1', 'у u1', "нее n' i1 j o0", "язвенная j a0 z v' e4 n n a4 e4", "болезнь b a1 l' e0 z' n'", 'почувствовала p a1 ch u0 s t u4 v u4 l o4', "улучшение u1 l u1 ch_ sh e0 n' i4 e4", 'стала s t a0 l a4', "вскоре f s k o0 r' e4", "ходить h a1 d' i0 t'", "октябре y1 k t' i1 b r' e0", "отправили a1 t p r 

In [72]:
with open(path+'/lexicon.txt', 'w', encoding="utf-8") as fl:
    for k, v in new_res_dict.items():
      fl.write(k + ' ' + " ".join([l[0] if len(l) > 1 and (l[1] == '4' or l[1] == '1') else l for l in v]) +"\n")

In [73]:
with open(path+'/lexicon.txt', "r", encoding=detect_encoding(path + '/lexicon.txt')) as f:
  lines = [i.strip() for i in f.readlines()]
  print(lines)

["юрий j u0 r' i", "трифонов t r' i0 f a n a f", "обмен a b m' e0 n", 'в f', "июле y j u0 l' e", "мать m a0 t'", "дмитриева d m' i0 t r' i i v a", "ксения k s' e0 n' i i", "тяжело t' i zh y l o0", "заболела z a2 b a l' e0 l a", 'и i', 'ее j i j o0', "отвезли a t v' i z l' i0", "боткинскую b o0 t k' i n s k u j u", 'она a n a0', "пролежала p r a2 l' i zh a0 l a", "двенадцать d v' i n a0 c y t'", "дней d n' e0 i", 'на n a', 'самое s a0 m a e', 'худшее h u0 t sh e', "сентябре s' i n t' i b r' e0", "сделали z' d' e0 l y l' e", "операцию a p' e r a0 c y y", "подтвердилось p y t v' i r d' i0 l y s'", 'но n o', 'считавшая sc i t a0 f sh e e', 'что sh t o', 'у u', "нее n' i j o0", "язвенная j a0 z v' e n n a e", "болезнь b a l' e0 z' n'", 'почувствовала p a ch u0 s t u v u l o', "улучшение u l u ch_ sh e0 n' i e", 'стала s t a0 l a', "вскоре f s k o0 r' e", "ходить h a d' i0 t'", "октябре y k t' i b r' e0", "отправили a t p r a0 v' i l' i", 'домой d a m o0 j', "твердо t v' o0 r d y", 'том t o0

In [155]:
with open(path+'/lexicon_stressless.txt', 'w', encoding="utf-8") as fl:
    for k, v in new_res_dict.items():
      fl.write(k + ' ' + " ".join([l[0] if len(l) > 1 and (l[1] == '4' or l[1] == '0' or l[1] == '1') else l for l in v]) +"\n")

In [158]:
with open(path+'/lexicon_stressless.txt', "r", encoding=detect_encoding(path + '/lexicon_stressless.txt')) as f:
  lines = [i.strip() for i in f.readlines()]
  print(lines)

["юрий j u r' i", "трифонов t r' i f a n a f", "обмен a b m' e n", 'в f', "июле y j u l' e", "мать m a t'", "дмитриева d m' i t r' i i v a", "ксения k s' e n' i i", "тяжело t' i zh y l o", "заболела z a2 b a l' e l a", 'и i', 'ее j i j o', "отвезли a t v' i z l' i", "боткинскую b o t k' i n s k u j u", 'она a n a', "пролежала p r a2 l' i zh a l a", "двенадцать d v' i n a c y t'", "дней d n' e i", 'на n a', 'самое s a m a e', 'худшее h u t sh e', "сентябре s' i n t' i b r' e", "сделали z' d' e l y l' e", "операцию a p' e r a c y y", "подтвердилось p y t v' i r d' i l y s'", 'но n o', 'считавшая sc i t a f sh e e', 'что sh t o', 'у u', "нее n' i j o", "язвенная j a z v' e n n a e", "болезнь b a l' e z' n'", 'почувствовала p a ch u s t u v u l o', "улучшение u l u ch_ sh e n' i e", 'стала s t a l a', "вскоре f s k o r' e", "ходить h a d' i t'", "октябре y k t' i b r' e", "отправили a t p r a v' i l' i", 'домой d a m o j', "твердо t v' o r d y", 'том t o m', "дело d' e l a", 'вот v o t', "

# Префиксное дерево

In [ ]:
class Leaf: # Префиксное дерево
  def __init__(self, prefix, word = None):
    self.prefix = prefix
    self.word = [] 
    if word:
      self.word.append(word)
    self.leaves = {}
  
  def __repr__(self):
    return f'{self.prefix}, {self.word}, {self.leaves}'

def create_tree(language_model):
  root = Leaf([])

  for k, v in language_model.items():
    current = root
    for s, _ in enumerate(v):
      if v[s] not in current.leaves:
        current.leaves[v[s]] = Leaf(v[:s + 1])
      current = current.leaves[v[s]]
    current.word.append(k)

  return root

In [ ]:
tree = create_tree(new_res_dict)

In [ ]:
tree

[], [], {'j': ['j'], ['ехать'], {'u0': ['j', 'u0'], [], {"r'": ['j', 'u0', "r'"], [], {'i4': ['j', 'u0', "r'", 'i4'], ['юрий'], {}}, 'p': ['j', 'u0', 'p'], [], {"k'": ['j', 'u0', 'p', "k'"], [], {'i4': ['j', 'u0', 'p', "k'", 'i4'], ['юбке'], {}}, 'k': ['j', 'u0', 'p', 'k'], [], {'u4': ['j', 'u0', 'p', 'k', 'u4'], ['юбку'], {}}}, 'n': ['j', 'u0', 'n'], [], {'y4': ['j', 'u0', 'n', 'y4'], [], {'sh': ['j', 'u0', 'n', 'y4', 'sh'], [], {'y4': ['j', 'u0', 'n', 'y4', 'sh', 'y4'], ['юноша'], {}}}}, 'k': ['j', 'u0', 'k'], ['юг'], {}, 'g': ['j', 'u0', 'g'], [], {'e4': ['j', 'u0', 'g', 'e4'], ['юго-'], {}}}, 'i1': ['j', 'i1'], [], {'j': ['j', 'i1', 'j'], [], {'o0': ['j', 'i1', 'j', 'o0'], ['ее'], {}}, 'sc': ['j', 'i1', 'sc'], [], {'o0': ['j', 'i1', 'sc', 'o0'], ['еще'], {}}, 'v': ['j', 'i1', 'v'], [], {"l'": ['j', 'i1', 'v', "l'"], [], {'a0': ['j', 'i1', 'v', "l'", 'a0'], [], {'c': ['j', 'i1', 'v', "l'", 'a0', 'c'], [], {'y4': ['j', 'i1', 'v', "l'", 'a0', 'c', 'y4'], ['являться'], {}}, 'f': ['j', 

# Обучаем фонетический транскриптор

Фонетическая транскрипция (G2P)

В репозитории ссылка на пакет и ноутбук

Пакету для работы нужен файл с транскрипциями , на которых он будет обучаться 
(взять тот, который делали на корпрессе), например:

ма1ма m a0 m a

...

Обучить и проверить на spont.txt (транскрипция в spont_trans.txt)

Там разные обозначения могут быть, поэтому нужно установить соответствие

## Обучаем с ударениями

In [74]:
path+'/lexicon.txt'

'/content/drive/MyDrive/STUDY_MY/Programming/cta_seg/lexicon.txt'

In [1]:
!pip install phonetisaurus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 70.9 MB/s eta 0:00:00


In [88]:
import phonetisaurus

In [75]:
!phonetisaurus train --model model.fst /content/drive/MyDrive/STUDY_MY/Programming/cta_seg/lexicon.txt

INFO:phonetisaurus-train:2023-04-17 07:03:29:  Checking command configuration...
DEBUG:phonetisaurus-train:2023-04-17 07:03:29:  Directory does not exist.  Trying to create.
INFO:phonetisaurus-train:2023-04-17 07:03:29:  Checking lexicon for reserved characters: '}', '|', '_'...
DEBUG:phonetisaurus-train:2023-04-17 07:03:29:  arpa_path:  train/model.o8.arpa
DEBUG:phonetisaurus-train:2023-04-17 07:03:29:  corpus_path:  train/model.corpus
DEBUG:phonetisaurus-train:2023-04-17 07:03:29:  dir_prefix:  train
DEBUG:phonetisaurus-train:2023-04-17 07:03:29:  grow:  False
DEBUG:phonetisaurus-train:2023-04-17 07:03:29:  lexicon_file:  /tmp/tmpk1u1a6ih.txt
DEBUG:phonetisaurus-train:2023-04-17 07:03:29:  logger:  <Logger phonetisaurus-train (DEBUG)>
DEBUG:phonetisaurus-train:2023-04-17 07:03:29:  makeJointNgramCommand:  <bound method G2PModelTrainer._mitlm of <__main__.G2PModelTrainer object at 0x7f5261b80790>>
DEBUG:phonetisaurus-train:2023-04-17 07:03:29:  model_path:  train/model.fst
DEBUG:phone

In [76]:
!phonetisaurus predict --model model.fst апстену

апстену a p s' t' e0 n u


In [123]:
path1 = '/content/drive/MyDrive/STUDY_MY/Programming/stress/'

In [124]:
spont = read_txt(path1 + 'spont.txt')
spont_trans = read_txt(path1 + 'spont_trans.txt')

In [125]:
spont_trans

[['ta0k',
  "smatr'i0",
  'u',
  'na0s',
  "t'ip'e0r'",
  "ka0rtač'k'i",
  'na0',
  'kato0rɨx',
  'nu0žna',
  'maršru0d',
  'da0',
  "pralažɨ0t'"],
 ['a0'],
 ['da0', "b'ir'i0"],
 ["zna0č'it",
  "p'e0rvai",
  "ka0rtač'ka",
  'u',
  "m'in'a0",
  "smatr'i0",
  'u',
  'na0s',
  'tut',
  "jiš'o0",
  'mo0gut',
  "razl'ič'a0ca",
  "sa0m'i",
  "ab'je0ktɨ"],
 ['nu0', "pr'im'e0rna", "an'i0", 'bu0dut', "ad'ina0kavɨi", 'da0'],
 ["zna0č'it", "p'e0rvai", "ka0rtač'ka", 'u', "m'in'a0", 'vaje0nnɨj', 'rajo0n'],
 ["je0z'd'", 'da0'],
 ['o0n', 'u', "t'ib'a0", "gd'e0", "naxo0d'ica", 'na', "ka0rtač'k'i"],
 ["l'e0vɨj", 'u0gal'],
 ['vo0t', "smatr'i0", 'tɨ0', "bu0d'iš", "s'ič'a0š'", "č'irt'i0t'"],
 ['karandašo0m', 'na0', 'svaje0j', "ka0rtač'k'i"],
 ["zna0č'it", 'vo0t', 'po0d', 'vaje0nnɨm', 'rajo0nam'],
 ['po0d',
  'vaje0nnɨm',
  'rajo0nam',
  'vo0t',
  'o0n',
  'u',
  "t'ib'a0",
  'v',
  "l'e0vam",
  'uglu0',
  "n'i0žn'im",
  'da0'],
 ['tɨ0',
  "č'ut'-č'u0t'",
  'ad',
  'bu0kvɨ',
  've0',
  "zna0č'it",
  'tɨ0',

In [126]:
def change_marks(texts):
  for i, v in enumerate(texts):
    for j, k in enumerate(v):
      texts[i][j] = texts[i][j].replace("č", "ch")
      texts[i][j] = texts[i][j].replace("ɨ", "y")
      texts[i][j] = texts[i][j].replace("š", "sh")
      texts[i][j] = texts[i][j].replace("ž", "zh")
  return texts

In [127]:
spont_trans = change_marks(spont_trans)

In [128]:
def clear_stress(texts):
  new_texts = []
  for i in texts:
    line = []
    for k in i:
      line.append(k.replace('1', ''))
    new_texts.append(line)
  return new_texts

In [129]:
spont_without_stress = clear_stress(spont)
print(spont_without_stress)

[['так', 'смотри', 'у', 'нас', 'теперь', 'карточки', 'на', 'которых', 'нужно', 'маршрут', 'да', 'проложить'], ['а'], ['да', 'бери'], ['значит', 'первая', 'карточка', 'у', 'меня', 'смотри', 'у', 'нас', 'тут', 'ещё', 'могут', 'различаться', 'сами', 'объекты'], ['ну', 'примерно', 'они', 'будут', 'одинаковые', 'да'], ['значит', 'первая', 'карточка', 'у', 'меня', 'военный', 'район'], ['есть', 'да'], ['он', 'у', 'тебя', 'где', 'находится', 'на', 'карточке'], ['левый', 'угол'], ['вот', 'смотри', 'ты', 'будешь', 'сейчас', 'чертить'], ['карандашом', 'на', 'своей', 'карточке'], ['значит', 'вот', 'под', 'военным', 'районом'], ['под', 'военным', 'районом', 'вот', 'он', 'у', 'тебя', 'в', 'левом', 'углу', 'нижнем', 'да'], ['ты', 'чуть-чуть', 'от', 'буквы', 'вэ', 'значит', 'ты', 'ты', 'от', 'буквы', 'вэ', 'сантиметр', 'вниз', 'отступаешь'], ['вниз'], ['от', 'края', 'буквы', 'вэ', 'и', 'по2лсантиметра', 'влево'], ['вот', 'это', 'у', 'тебя', 'старт'], ['вот', 'точку', 'поставил'], ['вот', 'под', 'военн

In [132]:
spont_transcribed_by_g2p = [["".join(k[1]) for k in phonetisaurus.predict(i, 'model.fst')] for i in spont_without_stress]

In [133]:
spont_transcribed_by_g2p

[['ta0k',
  "smatr'i0",
  'u',
  'na0s',
  "t'ip'e0r'",
  "kartychk'i",
  'na',
  'kato0ryh',
  'nu0zhny',
  'ma0rshru0t',
  'da0',
  "pra2lazhy0t'"],
 ['a'],
 ['da0', "b'ir'i"],
 ['zna0chit',
  "p'e0rvee",
  'kartychka',
  'u',
  "m'in'a0",
  "smatr'i0",
  'u',
  'na0s',
  'tu0t',
  'jisc',
  'mo0gut',
  "ryzl'icha0ca",
  "sa0m'i",
  "ab'je0kty"],
 ['nu', "pr'im'e0rna", "an'i0", 'bu0duc', "ad'ina0kuvyy", 'da0'],
 ['zna0chit', "p'e0rvee", 'kartychka', 'u', "m'in'a0", 'vyenyj', 'ra0io0n'],
 ["is'", 'da0'],
 ['o0n', 'u', "t'ib'a0", "gd'e", "naho0d'icy", 'na', "kartychk'i"],
 ["l'e0vyj", 'u0gyl'],
 ['vo0t', "smatr'i0", 'ty', "bu0d'ish", "s'icha0s", "chirt'i0t'"],
 ['karanda0shom', 'na', 'svaje0j', "kartychk'i"],
 ['zna0chit', 'vo0t', 'po0t', 'vyenym', 'ra0io0nam'],
 ['po0t',
  'vyenym',
  'ra0io0nam',
  'vo0t',
  'o0n',
  'u',
  "t'ib'a0",
  'f',
  "l'e0vam",
  'uglu0',
  "n'i0zhn'em",
  'da0'],
 ['ty',
  'chu0tchu0t',
  'at',
  'bu0kvy0',
  've0',
  'zna0chit',
  'ty',
  'ty',
  'at',
  

In [120]:
def damerau_levenshtein_distance(a, b):
  a = '\'' + a
  b = '\'' + b
  m = len(a)
  n = len(b)
  matrix = [[0 for i in range(n)] for i in range(m)]

  for i in range(1, m):
    matrix[i][0] = i
  
  for j in range(1, n):
    matrix[0][j] = j

  for i in range(1, m):
    for j in range(1, n):
      substitution_cost = 0
      if a[i] != b[j]:
        substitution_cost = 1
      matrix[i][j] = min([matrix[i-1][j] + 1,                   # deletion
                         matrix[i][j-1] + 1,                    # insertion
                         matrix[i-1][j-1] + substitution_cost]) # substitution
      
      if i > 1 and j > 1 and a[i] == b[j-1] and a[i-1] == b[j]:
        matrix[i][j] = min(matrix[i][j],
                            matrix[i - 2][j - 2] + 1)           # transposition
    
  return matrix[m - 1][n - 1]

In [147]:
def levenshtein_distance(a, b):
  a = '\'' + a
  b = '\'' + b
  m = len(a)
  n = len(b)
  matrix = [[0 for i in range(n)] for i in range(m)]

  for i in range(1, m):
    matrix[i][0] = i
  
  for j in range(1, n):
    matrix[0][j] = j

  for i in range(1, m):
    for j in range(1, n):
      substitution_cost = 0
      if a[i] != b[j]:
        substitution_cost = 1
      matrix[i][j] = min([matrix[i-1][j] + 1,                   # deletion
                         matrix[i][j-1] + 1,                    # insertion
                         matrix[i-1][j-1] + substitution_cost]) # substitution
  #for i in matrix:
  #  print(' '.join(str(i)))
    
  return matrix[m - 1][n - 1]

In [154]:
sum_of_cers = 0
count = 0
for i, s in enumerate(spont_trans):
  for k, v in enumerate(s):
    spont_trans[i][k]
    try:
      sum_of_cers += levenshtein_distance(spont_trans[i][k], spont_transcribed_by_g2p[i][k])
      count += 1
    except Exception as e:
      print(e)
      sum_of_cers += len(spont_trans[i][k])
      count += 1
print("CER - " + str(sum_of_cers / count)) # Средняя CER

list index out of range
CER - 1.1860909438288116


## Обучаем без ударений

In [159]:
!phonetisaurus train --model model2.fst /content/drive/MyDrive/STUDY_MY/Programming/cta_seg/lexicon_stressless.txt

INFO:phonetisaurus-train:2023-04-17 08:42:18:  Checking command configuration...
DEBUG:phonetisaurus-train:2023-04-17 08:42:18:  Directory does not exist.  Trying to create.
INFO:phonetisaurus-train:2023-04-17 08:42:18:  Checking lexicon for reserved characters: '}', '|', '_'...
DEBUG:phonetisaurus-train:2023-04-17 08:42:18:  arpa_path:  train/model.o8.arpa
DEBUG:phonetisaurus-train:2023-04-17 08:42:18:  corpus_path:  train/model.corpus
DEBUG:phonetisaurus-train:2023-04-17 08:42:18:  dir_prefix:  train
DEBUG:phonetisaurus-train:2023-04-17 08:42:18:  grow:  False
DEBUG:phonetisaurus-train:2023-04-17 08:42:18:  lexicon_file:  /tmp/tmpv7hvnjhe.txt
DEBUG:phonetisaurus-train:2023-04-17 08:42:18:  logger:  <Logger phonetisaurus-train (DEBUG)>
DEBUG:phonetisaurus-train:2023-04-17 08:42:18:  makeJointNgramCommand:  <bound method G2PModelTrainer._mitlm of <__main__.G2PModelTrainer object at 0x7f425968d790>>
DEBUG:phonetisaurus-train:2023-04-17 08:42:18:  model_path:  train/model.fst
DEBUG:phone

In [160]:
spont_transcribed_by_g2p_stressless = [["".join(k[1]) for k in phonetisaurus.predict(i, 'model2.fst')] for i in spont_without_stress]

In [163]:
sum_of_cers = 0
count = 0
for i, s in enumerate(spont_trans):
  for k, v in enumerate(s):
    spont_trans[i][k]
    try:
      sum_of_cers += levenshtein_distance(spont_trans[i][k].replace('0', ''), spont_transcribed_by_g2p_stressless[i][k])
      count += 1
    except Exception as e:
      print(e)
      sum_of_cers += len(spont_trans[i][k])
      count += 1
print("CER - " + str(sum_of_cers / count)) # Средняя CER с g2p без учета ударений

list index out of range
CER - 0.7609858616736721
